<h1 style="text-align: center;">Regras (decision tree) - CASH-OUT

## 

In [1]:
prob = 0
smote = 0.0
split = True

### Cenários

In [2]:
lst_cenarios = [
    f'sem prob',
    f'prob',
    f'prob_bin {prob:2%}',
]

### Configs

In [3]:
## bibliotecas
import time
tic = time.time()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

from sklearn.metrics import *
from scipy.stats import * 

from utils.dspipe.helpers.rule_helper import RuleHelper
from utils.eda import *
from utils.modelling_tools import *

%load_ext autoreload
%autoreload 2

pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

seed = 123

### Dataset

In [4]:
file_path = f'dados/df_cashout_probs.parquet'
file_path_og = 'dados/df_cashout_treino_feature_eng.parquet'

## Carregando dataset
df_preds = pd.read_parquet(file_path)
df_og = pd.read_parquet(file_path_og)


In [5]:
df_og.columns

Index(['transaction_id', 'transaction_amount_total',
       'transaction_datetime_request', 'last_status',
       'transaction_payer_registration_data_doc',
       'transaction_receiver_registration_data_doc', 'fraud', 'hora', 'dia',
       'dia_semana', 'test_transaction', 'sum_amount_same_doc_in_1H',
       'sum_amount_same_doc_in_7D', 'sum_amount_same_doc_in_30D',
       'count_amount_same_doc_in_1H', 'count_amount_same_doc_in_7D',
       'count_amount_same_doc_in_30D', 'mean_amount_same_doc_in_1H',
       'mean_amount_same_doc_in_7D', 'mean_amount_same_doc_in_30D',
       'std_amount_in_1H', 'std_amount_in_7D', 'std_amount_in_30D',
       'amount_7D_1H', 'amount_30d_7d', 'behav_increase_7d_1H_amount',
       'behav_increase_30d_7d_amount', 'transacoes_unicas_pagador_recebedor',
       'recebedores_unicos_por_pagador', 'pagadores_unicos_por_recebedor',
       'razao_valor_frequencia_1H', 'razao_valor_frequencia_7D',
       'razao_valor_frequencia_30D', 'roc_razao_valor_frequencia_1H

In [6]:
df_og['rejected'] = np.where(df_og['last_status'] == 'REJECTED', 1, 0 )

feature_names = ['transaction_id', 'transaction_datetime_request',                
      'transaction_amount_total', 'sum_amount_same_doc_in_1H',
      'sum_amount_same_doc_in_7D', 'sum_amount_same_doc_in_30D',
      'count_amount_same_doc_in_1H', 'count_amount_same_doc_in_7D',
      'count_amount_same_doc_in_30D', 'mean_amount_same_doc_in_1H',
      'mean_amount_same_doc_in_7D', 'mean_amount_same_doc_in_30D',
      'std_amount_in_1H', 'std_amount_in_7D', 'std_amount_in_30D',
      'amount_7D_1H', 'amount_30d_7d', 'behav_increase_7d_1H_amount',
      'behav_increase_30d_7d_amount', 'flag_first_time_receiver',
      'desvio_relativo_media_in_1H', 'desvio_relativo_media_in_7D',
      'desvio_relativo_media_in_30D', 'last_status', 'transacoes_unicas_pagador_recebedor',
      'recebedores_unicos_por_pagador', 'pagadores_unicos_por_recebedor',
      'razao_valor_frequencia_1H', 'razao_valor_frequencia_7D', 'razao_valor_frequencia_30D',
      'roc_razao_valor_frequencia_1H','roc_razao_valor_frequencia_7D', 'roc_razao_valor_frequencia_30D',
      ]

# feature_names = ['transaction_id', 'transaction_datetime_request',  'last_status', 
#        # 'transaction_amount_total', 'flag_first_time_receiver',
#        # 'sum_amount_same_doc_in_1H', 'sum_amount_same_doc_in_7D', 'sum_amount_same_doc_in_30D',
#        # 'count_amount_same_doc_in_1H', 'count_amount_same_doc_in_7D', 'count_amount_same_doc_in_30D', 
#        # 'mean_amount_same_doc_in_1H', 'mean_amount_same_doc_in_7D', 'mean_amount_same_doc_in_30D', 
#     #    'std_amount_in_1H', 'std_amount_in_7D', 'std_amount_in_30D',
#        'amount_7D_1H', 'amount_30d_7d', 'behav_increase_7d_1H_amount', 'behav_increase_30d_7d_amount', 
#        # 'desvio_relativo_media_in_1H', 'desvio_relativo_media_in_7D', 'desvio_relativo_media_in_30D',
#        'transacoes_unicas_pagador_recebedor', 'recebedores_unicos_por_pagador', 'pagadores_unicos_por_recebedor',
#        'razao_valor_frequencia_1H', 'razao_valor_frequencia_7D', 'razao_valor_frequencia_30D',
#     #    'roc_razao_valor_frequencia_1H', 'roc_razao_valor_frequencia_7D', 'roc_razao_valor_frequencia_30D',
#        ]


# feature_names = ['transaction_id', 'last_status', 'transaction_datetime_request',
# 'amount_7D_1H',
# 'count_amount_same_doc_in_30D',
# 'count_amount_same_doc_in_7D',
# 'desvio_relativo_media_in_1H',
# 'desvio_relativo_media_in_7D',
# 'mean_amount_same_doc_in_1H',
# 'mean_amount_same_doc_in_7D',
# 'razao_valor_frequencia_30D',
# 'sum_amount_same_doc_in_30D',
# 'sum_amount_same_doc_in_7D',
# 'transacoes_unicas_pagador_recebedor']

# feature_names = [ 'transaction_id', 'last_status', 'transaction_datetime_request',
# 'razao_valor_frequencia_7D',
# # 'desvio_relativo_media_in_30D',
# 'behav_increase_30d_7d_amount',
# 'mean_amount_same_doc_in_30D',
# 'sum_amount_same_doc_in_1H',
# 'amount_30d_7d',
# 'std_amount_in_1H',
# 'flag_first_time_receiver',
# 'rejected',
#                             ]

df = df_og[feature_names].sort_values('transaction_id')

#### Criando Variável Resposta

In [7]:
df = pd.merge(df, df_preds[['transaction_id', 'prob', 'fraud']], on='transaction_id', how='left')

df['fraud'] = np.where(df['last_status'].isin(['FRAUD', 'CONT_FRAUD']), 1, 0)

pd.concat([df['fraud'].value_counts().rename('counts'),
            df['fraud'].value_counts(normalize=True).rename('frequency')], axis=1)

,counts,frequency
fraud,,
0,10110696,1.0000
1,52,0.0000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10110748 entries, 0 to 10110747
Data columns (total 35 columns):
 #   Column                               Dtype         
---  ------                               -----         
 0   transaction_id                       string        
 1   transaction_datetime_request         datetime64[ns]
 2   transaction_amount_total             float64       
 3   sum_amount_same_doc_in_1H            float64       
 4   sum_amount_same_doc_in_7D            float64       
 5   sum_amount_same_doc_in_30D           float64       
 6   count_amount_same_doc_in_1H          int64         
 7   count_amount_same_doc_in_7D          int64         
 8   count_amount_same_doc_in_30D         int64         
 9   mean_amount_same_doc_in_1H           float64       
 10  mean_amount_same_doc_in_7D           float64       
 11  mean_amount_same_doc_in_30D          float64       
 12  std_amount_in_1H                     float64       
 13  std_amount_in_7D         

In [9]:
to_drop = [
# 'behav_increase_7d_1H_amount',
# 'behav_increase_30d_7d_amount',
# 'predictions',	
# 'predictions_cutoff',	
# 'predictions_cutoff_70',	
# 'transaction_amount_total',
# 'sum_amount_same_doc_in_1H',
# 'sum_amount_same_doc_in_7D',
# 'sum_amount_same_doc_in_30D',
# 'count_amount_same_doc_in_1H',
# 'count_amount_same_doc_in_7D',
# 'count_amount_same_doc_in_30D',
# 'mean_amount_same_doc_in_1H',
# 'mean_amount_same_doc_in_7D',
# 'mean_amount_same_doc_in_30D',
# 'std_amount_in_1H',
# 'std_amount_in_7D',
# 'std_amount_in_30D',
# 'amount_7D_1H',
# 'amount_30d_7d',
# 'payer_receiver',
# 'flag_first_time_receiver',
# 'desvio_relativo_media_in_1H',
# 'desvio_relativo_media_in_7D',
# 'desvio_relativo_media_in_30D',
# 'prob'
]

if prob > 0: 
    df['prob_bin'] = np.where(df['prob'] >= prob, 1, 0)
    to_drop.append('prob')
if prob < 0:
    to_drop.append('prob')
    
to_drop

[]

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10110748 entries, 0 to 10110747
Data columns (total 35 columns):
 #   Column                               Dtype         
---  ------                               -----         
 0   transaction_id                       string        
 1   transaction_datetime_request         datetime64[ns]
 2   transaction_amount_total             float64       
 3   sum_amount_same_doc_in_1H            float64       
 4   sum_amount_same_doc_in_7D            float64       
 5   sum_amount_same_doc_in_30D           float64       
 6   count_amount_same_doc_in_1H          int64         
 7   count_amount_same_doc_in_7D          int64         
 8   count_amount_same_doc_in_30D         int64         
 9   mean_amount_same_doc_in_1H           float64       
 10  mean_amount_same_doc_in_7D           float64       
 11  mean_amount_same_doc_in_30D          float64       
 12  std_amount_in_1H                     float64       
 13  std_amount_in_7D         

### Train-Test Split

In [11]:
df = df.sort_values('transaction_datetime_request')

if split:
    data_split = '2024-07-15'

    df_train = df[df['transaction_datetime_request'] < data_split]
    df_test = df[df['transaction_datetime_request'] >= data_split]

    ## Separando bases de Treino e Teste
    y_train = df_train['fraud']
    X_train_full = df_train.drop('fraud', axis=1)

    y_test = df_test['fraud']
    X_test_full = df_test.drop('fraud', axis=1)
    transaction_id_X_test = X_test_full['transaction_id']

    numerics = ['int16','int32','int64','float16','float32','float64']
    numeric_features = X_train_full.select_dtypes(include=numerics).columns
    X_train = X_train_full[numeric_features]
    X_test = X_test_full[numeric_features]

elif split == False:
    y = df['fraud']
    X = df.drop('fraud', axis=1)
    
    X_train, X_test = X, X
    y_train, y_test = y, y


if smote > 0 :
        smote = SMOTE(random_state=seed, sampling_strategy=smote)
        X_train_resampled,  y_train_resampled = smote.fit_resample(X_train, y_train)
elif smote <= 0 :
        X_train_resampled, y_train_resampled = X_train, y_train

In [12]:
# X_test_prob_50 = X_test[X_test['prob'] > 0.5]
# X_test = X_test[X_test['prob'] <= 0.5]
X_test.head()

,transaction_amount_total,sum_amount_same_doc_in_1H,sum_amount_same_doc_in_7D,sum_amount_same_doc_in_30D,count_amount_same_doc_in_1H,count_amount_same_doc_in_7D,count_amount_same_doc_in_30D,mean_amount_same_doc_in_1H,mean_amount_same_doc_in_7D,mean_amount_same_doc_in_30D,std_amount_in_1H,std_amount_in_7D,std_amount_in_30D,amount_7D_1H,amount_30d_7d,behav_increase_7d_1H_amount,behav_increase_30d_7d_amount,flag_first_time_receiver,desvio_relativo_media_in_1H,desvio_relativo_media_in_7D,desvio_relativo_media_in_30D,transacoes_unicas_pagador_recebedor,recebedores_unicos_por_pagador,pagadores_unicos_por_recebedor,razao_valor_frequencia_1H,razao_valor_frequencia_7D,razao_valor_frequencia_30D,roc_razao_valor_frequencia_1H,roc_razao_valor_frequencia_7D,roc_razao_valor_frequencia_30D,prob
2195602,20.0000,47950.4500,3995298.4400,18154704.4500,352,28350,134258,136.2229,140.9276,135.2225,742.9644,602.2391,564.9873,27998,105908,79.5398,0.7888,0,0.1468,0.1419,0.1479,134258,1118,538,133.6467,140.5244,134.8948,-0.0024,0.0000,-0.0000,0.0800
2195603,150.0000,150.0000,150.0000,200.0000,1,1,2,150.0000,150.0000,100.0000,0.0000,0.0000,70.7107,0,1,0.0000,0.5000,0,1.0000,1.0000,1.5000,2,96418,1,137.0762,213.2601,213.9926,0.0008,0.0000,-0.0000,0.0200
2195604,287.0000,48237.4500,3995585.4400,18154991.4500,353,28351,134259,136.6500,140.9328,135.2236,741.9517,602.2291,564.9853,27998,105908,79.3144,0.7888,0,2.1003,2.0364,2.1224,134259,1118,538,134.0703,140.5295,134.8959,0.0032,0.0000,0.0000,0.1100
2195605,180.4200,180.4200,664.5000,1356.4900,1,4,9,180.4200,166.1250,150.7211,0.0000,91.7721,111.1882,3,5,3.0000,0.5556,0,1.0000,1.0860,1.1970,9,9330,1,194.5471,298.2706,285.0774,-0.0013,-0.0005,0.0000,0.1300
2195606,75.0000,1769.5000,226859.8000,507992.1500,35,2829,7067,50.5571,80.1908,71.8823,128.0549,704.2309,489.9209,2794,4238,79.8286,0.5997,0,1.4835,0.9353,1.0434,7067,395,2,52.7027,106.2569,86.2252,0.0516,0.0006,-0.0002,0.4200


In [13]:
X_train_resampled.drop(to_drop, axis='columns', inplace=True)
X_test.drop(to_drop, axis='columns', inplace=True)


print(f"""X_train.shape: {X_train_resampled.shape}
y_train.shape: {y_train_resampled.shape} 
X_test.shape: {X_test.shape}
y_test.shape: {y_test.shape}
""")

X_train.shape: (2200098, 31)
y_train.shape: (2200098,) 
X_test.shape: (7910650, 31)
y_test.shape: (7910650,)



### Check final - Variáveis e Target

In [14]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2200098 entries, 3766880 to 2195601
Data columns (total 31 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   transaction_amount_total             float64
 1   sum_amount_same_doc_in_1H            float64
 2   sum_amount_same_doc_in_7D            float64
 3   sum_amount_same_doc_in_30D           float64
 4   count_amount_same_doc_in_1H          int64  
 5   count_amount_same_doc_in_7D          int64  
 6   count_amount_same_doc_in_30D         int64  
 7   mean_amount_same_doc_in_1H           float64
 8   mean_amount_same_doc_in_7D           float64
 9   mean_amount_same_doc_in_30D          float64
 10  std_amount_in_1H                     float64
 11  std_amount_in_7D                     float64
 12  std_amount_in_30D                    float64
 13  amount_7D_1H                         int64  
 14  amount_30d_7d                        int64  
 15  behav_increase_7d_1H_amount    

In [15]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7910650 entries, 2195602 to 10110709
Data columns (total 31 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   transaction_amount_total             float64
 1   sum_amount_same_doc_in_1H            float64
 2   sum_amount_same_doc_in_7D            float64
 3   sum_amount_same_doc_in_30D           float64
 4   count_amount_same_doc_in_1H          int64  
 5   count_amount_same_doc_in_7D          int64  
 6   count_amount_same_doc_in_30D         int64  
 7   mean_amount_same_doc_in_1H           float64
 8   mean_amount_same_doc_in_7D           float64
 9   mean_amount_same_doc_in_30D          float64
 10  std_amount_in_1H                     float64
 11  std_amount_in_7D                     float64
 12  std_amount_in_30D                    float64
 13  amount_7D_1H                         int64  
 14  amount_30d_7d                        int64  
 15  behav_increase_7d_1H_amount   

In [16]:
# df_test[df_test['transaction_datetime_request'] > data_split]['roc_trazao_valor_frequencia'].describe()

In [17]:
y_train_resampled.value_counts()

fraud
0    2200075
1         23
Name: count, dtype: int64

In [18]:
y_test.value_counts()

fraud
0    7910621
1         29
Name: count, dtype: int64

### Decision Tree Model

In [19]:
## Decision Tree
clf = RandomForestClassifier(max_depth=5, random_state=seed)
clf.fit(X_train_resampled, y_train_resampled)

y_pred = clf.predict(X_test)

# plt.figure(figsize=(60, 10), dpi=300)
# plot_tree(clf, 
#         feature_names= X_train.columns, 
#         class_names=["good", "FRAUD"], 
#         filled=True, 
#         rounded=True, 
#         fontsize=10)

# description = f"""Cenário: {(lst_cenarios[2])}
# Casos teste: {y_test.shape}
# Fraudes Teste: {y_test.value_counts()[1]}"""

# plt.text(0.5, 0.98, description, wrap=True, horizontalalignment='center', fontsize=12)

# plt.show()

In [20]:
# X_test

In [21]:
y_test

2195602     0
2195603     0
2195604     0
2195605     0
2195606     0
           ..
10110647    0
10110706    0
10110661    0
10110650    0
10110709    0
Name: fraud, Length: 7910650, dtype: int32

In [22]:
df_rf = pd.concat([X_test, y_test], axis='columns')
df_rf

,transaction_amount_total,sum_amount_same_doc_in_1H,sum_amount_same_doc_in_7D,sum_amount_same_doc_in_30D,count_amount_same_doc_in_1H,count_amount_same_doc_in_7D,count_amount_same_doc_in_30D,mean_amount_same_doc_in_1H,mean_amount_same_doc_in_7D,mean_amount_same_doc_in_30D,std_amount_in_1H,std_amount_in_7D,std_amount_in_30D,amount_7D_1H,amount_30d_7d,behav_increase_7d_1H_amount,behav_increase_30d_7d_amount,flag_first_time_receiver,desvio_relativo_media_in_1H,desvio_relativo_media_in_7D,desvio_relativo_media_in_30D,transacoes_unicas_pagador_recebedor,recebedores_unicos_por_pagador,pagadores_unicos_por_recebedor,razao_valor_frequencia_1H,razao_valor_frequencia_7D,razao_valor_frequencia_30D,roc_razao_valor_frequencia_1H,roc_razao_valor_frequencia_7D,roc_razao_valor_frequencia_30D,prob,fraud
2195602,20.0000,47950.4500,3995298.4400,18154704.4500,352,28350,134258,136.2229,140.9276,135.2225,742.9644,602.2391,564.9873,27998,105908,79.5398,0.7888,0,0.1468,0.1419,0.1479,134258,1118,538,133.6467,140.5244,134.8948,-0.0024,0.0000,-0.0000,0.0800,0
2195603,150.0000,150.0000,150.0000,200.0000,1,1,2,150.0000,150.0000,100.0000,0.0000,0.0000,70.7107,0,1,0.0000,0.5000,0,1.0000,1.0000,1.5000,2,96418,1,137.0762,213.2601,213.9926,0.0008,0.0000,-0.0000,0.0200,0
2195604,287.0000,48237.4500,3995585.4400,18154991.4500,353,28351,134259,136.6500,140.9328,135.2236,741.9517,602.2291,564.9853,27998,105908,79.3144,0.7888,0,2.1003,2.0364,2.1224,134259,1118,538,134.0703,140.5295,134.8959,0.0032,0.0000,0.0000,0.1100,0
2195605,180.4200,180.4200,664.5000,1356.4900,1,4,9,180.4200,166.1250,150.7211,0.0000,91.7721,111.1882,3,5,3.0000,0.5556,0,1.0000,1.0860,1.1970,9,9330,1,194.5471,298.2706,285.0774,-0.0013,-0.0005,0.0000,0.1300,0
2195606,75.0000,1769.5000,226859.8000,507992.1500,35,2829,7067,50.5571,80.1908,71.8823,128.0549,704.2309,489.9209,2794,4238,79.8286,0.5997,0,1.4835,0.9353,1.0434,7067,395,2,52.7027,106.2569,86.2252,0.0516,0.0006,-0.0002,0.4200,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10110647,50.0000,50.0000,50.0000,50.0000,1,1,1,50.0000,50.0000,50.0000,0.0000,0.0000,0.0000,0,0,0.0000,0.0000,1,1.0000,1.0000,1.0000,1,160367,1,213.7259,207.7118,178.6436,-0.0007,-0.0000,0.0000,0.0000,0
10110706,0.0100,0.0100,0.0100,0.0100,1,1,1,0.0100,0.0100,0.0100,0.0000,0.0000,0.0000,0,0,0.0000,0.0000,1,1.0000,1.0000,1.0000,1,1387,1,0.0100,236.7382,202.1934,0.0000,-0.0014,-0.0004,0.1700,0
10110661,0.0100,0.0100,0.0100,0.0100,1,1,1,0.0100,0.0100,0.0100,0.0000,0.0000,0.0000,0,0,0.0000,0.0000,1,1.0000,1.0000,1.0000,1,37254,1,0.0100,0.0100,0.0100,0.0000,0.0000,0.0000,0.1700,0
10110650,36.0000,36.0000,36.0000,36.0000,1,1,1,36.0000,36.0000,36.0000,0.0000,0.0000,0.0000,0,0,0.0000,0.0000,1,1.0000,1.0000,1.0000,1,55538,2,125.9741,509.1474,843.2240,0.0004,-0.0000,-0.0000,0.0000,0


In [23]:
ignore = [
    'prob'
]


In [24]:
n_rules=20
scoring = 'f1'
print(
    RuleHelper.get_top_rules(
                            "df_rf",
                            clf,
                            df_rf.columns,
                            max_depth=3,
                            n_rules=n_rules,
                            df_use_on_candidates=df_rf,
                            target_col="fraud",
                            candidates_scoring_func=scoring,
                            max_candidates=200,
                            max_activation_chance=0.05,
                            features_to_ignore=ignore)
    )

df_rf["rule118"] = np.where((df_rf["pagadores_unicos_por_recebedor"] > 530.5)&(df_rf["count_amount_same_doc_in_30D"] <= 448.5), 1, 0) # [[0.9991 0.0009]] score = 0.00

df_rf["rule153"] = np.where((df_rf["recebedores_unicos_por_pagador"] <= 22.5)&(df_rf["mean_amount_same_doc_in_1H"] > 477.5168151855469), 1, 0) # [[0.9998 0.0002]] score = 0.00

df_rf["rule91"] = np.where((df_rf["roc_razao_valor_frequencia_1H"] > 80.9776840209961), 1, 0) # [[0.9977 0.0023]] score = 0.00

df_rf["rule191"] = np.where((df_rf["recebedores_unicos_por_pagador"] > 5.5)&(df_rf["roc_razao_valor_frequencia_30D"] > 0.004699976881965995), 1, 0) # [[0.9999 0.0001]] score = 0.00

df_rf["rule123"] = np.where((df_rf["roc_razao_valor_frequencia_7D"] > 0.031195289455354214)&(df_rf["sum_amount_same_doc_in_7D"] <= 275905.9375), 1, 0) # [[0.9994 0.0006]] score = 0.00

df_rf["rule104"] = np.where((df_rf["roc_razao_valor_frequencia_7D"] > 0.031195289455354214), 1, 0) # [[0.9987 0.0013]] score = 0.00

df_rf["rule138"] = np.where

In [26]:
df_rf["rule118"] = np.where((df_rf["pagadores_unicos_por_recebedor"] > 530.5)&(df_rf["count_amount_same_doc_in_30D"] <= 448.5), 1, 0) # [[0.9991 0.0009]] score = 0.00

df_rf["rule153"] = np.where((df_rf["recebedores_unicos_por_pagador"] <= 22.5)&(df_rf["mean_amount_same_doc_in_1H"] > 477.5168151855469), 1, 0) # [[0.9998 0.0002]] score = 0.00

df_rf["rule91"] = np.where((df_rf["roc_razao_valor_frequencia_1H"] > 80.9776840209961), 1, 0) # [[0.9977 0.0023]] score = 0.00

df_rf["rule191"] = np.where((df_rf["recebedores_unicos_por_pagador"] > 5.5)&(df_rf["roc_razao_valor_frequencia_30D"] > 0.004699976881965995), 1, 0) # [[0.9999 0.0001]] score = 0.00

df_rf["rule123"] = np.where((df_rf["roc_razao_valor_frequencia_7D"] > 0.031195289455354214)&(df_rf["sum_amount_same_doc_in_7D"] <= 275905.9375), 1, 0) # [[0.9994 0.0006]] score = 0.00

df_rf["rule104"] = np.where((df_rf["roc_razao_valor_frequencia_7D"] > 0.031195289455354214), 1, 0) # [[0.9987 0.0013]] score = 0.00

df_rf["rule138"] = np.where((df_rf["roc_razao_valor_frequencia_7D"] > 0.010834779590368271)&(df_rf["amount_7D_1H"] <= 742.5), 1, 0) # [[0.9997 0.0003]] score = 0.00

df_rf["rule122"] = np.where((df_rf["roc_razao_valor_frequencia_7D"] > 0.010834779590368271), 1, 0) # [[0.9994 0.0006]] score = 0.00

df_rf["rule198"] = np.where((df_rf["recebedores_unicos_por_pagador"] <= 22.5), 1, 0) # [[0.9999 0.0001]] score = 0.00

df_rf["rule116"] = np.where((df_rf["desvio_relativo_media_in_30D"] <= 71.57551193237305)&(df_rf["roc_razao_valor_frequencia_1H"] > 3.185852885246277), 1, 0) # [[0.999 0.001]] score = 0.00

df_rf["rule139"] = np.where((df_rf["roc_razao_valor_frequencia_30D"] > 0.0009226450056303293)&(df_rf["razao_valor_frequencia_1H"] <= 24998.0048828125), 1, 0) # [[0.9997 0.0003]] score = 0.00

df_rf["rule92"] = np.where((df_rf["roc_razao_valor_frequencia_7D"] > 0.0751725509762764), 1, 0) # [[0.9977 0.0023]] score = 0.00

df_rf["rule93"] = np.where((df_rf["roc_razao_valor_frequencia_7D"] > 0.0751725509762764), 1, 0) # [[0.9979 0.0021]] score = 0.00

df_rf["rule127"] = np.where((df_rf["roc_razao_valor_frequencia_30D"] > 0.0009226450056303293), 1, 0) # [[0.9995 0.0005]] score = 0.00

df_rf["rule105"] = np.where((df_rf["desvio_relativo_media_in_7D"] <= 49.75229454040527)&(df_rf["roc_razao_valor_frequencia_1H"] > 7.05854058265686), 1, 0) # [[0.9987 0.0013]] score = 0.00

df_rf["rule142"] = np.where((df_rf["roc_razao_valor_frequencia_1H"] > 7.05854058265686)&(df_rf["roc_razao_valor_frequencia_1H"] > 7.061068296432495), 1, 0) # [[0.9997 0.0003]] score = 0.00

df_rf["rule117"] = np.where((df_rf["roc_razao_valor_frequencia_1H"] > 7.05854058265686), 1, 0) # [[0.9991 0.0009]] score = 0.00

df_rf["rule179"] = np.where((df_rf["roc_razao_valor_frequencia_30D"] > 0.0007949408318381757)&(df_rf["amount_30d_7d"] <= 239.5), 1, 0) # [[0.9999 0.0001]] score = 0.00

df_rf["rule95"] = np.where((df_rf["roc_razao_valor_frequencia_7D"] > 0.06972875073552132), 1, 0) # [[0.998 0.002]] score = 0.00

df_rf["rule140"] = np.where((df_rf["razao_valor_frequencia_1H"] <= 24998.0048828125)&(df_rf["roc_razao_valor_frequencia_7D"] > 0.003416219260543585), 1, 0) # [[0.9997 0.0003]] score = 0.00

In [27]:
rules_cols = []
for col in df_rf.columns:
    if "rule" in col: rules_cols.append(col)

In [28]:
res, _ = RuleHelper.get_tabular_rule_analysis(df_rf, rules_cols, value_col="prob", target_col="fraud")
res

┌─────────┬─────────────────────────┬────────────────────────┬─────────────┬────────────┬───────────────────┬─────────┬────────────────────┬──────────────────────────────┬──────────────────┬─────────────┬─────────────┬─────────────┐
│  rule   │ base_activation_percent │ percent_fraud_detected │ fp:tp-ratio │ percent_fp │ num_rows_affected │   fp    │ base_value_percent │ percent_value_fraud_detected │ percent_value_fp │ total_value │  avg_value  │  fp_value   │
│ varchar │         varchar         │        varchar         │   varchar   │  varchar   │      varchar      │ varchar │      varchar       │           varchar            │     varchar      │   varchar   │   varchar   │   varchar   │
├─────────┼─────────────────────────┼────────────────────────┼─────────────┼────────────┼───────────────────┼─────────┼────────────────────┼──────────────────────────────┼──────────────────┼─────────────┼─────────────┼─────────────┤
│ rule118 │ 0.128 %                 │ 34.483 %               │ 1015.

In [29]:
type(res)

duckdb.duckdb.DuckDBPyRelation